In [1]:
import pandas as pd
import requests
import numpy as np
from io import StringIO
import datetime

In [49]:
class TwStock:
#     對照表
    m_mapping = {'上市':'sii','上櫃':'otc'}
    base_url = 'https://mops.twse.com.tw/mops/web/'
    p_mapping = {'損益表':base_url+'t163sb04'
                ,'資產負債表':base_url+'t163sb05'
                ,'營益分析':base_url+'t163sb06'
                ,'財務結構分析':base_url+'t51sb02'}
    old_p_mapping = {'損益表':base_url+'t51sb08'
                ,'資產負債表':base_url+'t51sb07'
                ,'營益分析':base_url+'t51sb06'
                ,'財務結構分析':base_url+'t51sb02'}

    def __init__(self, year, season, mkt_type, purpose, \
                 filter_ind=True, m_mapping=m_mapping, p_mapping=p_mapping, old_p_mapping=old_p_mapping):
        year = year if year < 1000 else year-1911
        self.year = year
        self.season = '0'+str(season)
        self.mkt_type = m_mapping[mkt_type] if mkt_type in m_mapping else None
        self.purpose = purpose
        p_mapping = p_mapping if self.year>=102 else old_p_mapping
        self.url = p_mapping[self.purpose] if self.purpose in p_mapping else None
        self.filter_ind=filter_ind
    
    def add_raw(self):
        form = {'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':self.mkt_type,
            'year':self.year,
            'season':self.season,
            'ifrs':'Y'
            }
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            
        r = requests.post(self.url, form, headers=headers)
        r.encoding = 'utf8'
        dfs = pd.read_html(StringIO(r.text))
        self.raw_data = dfs
        
#     損益表
    def unify_incm(self):
        self.dfs = [i for i in self.raw_data if '公司代號' in i.columns]
        self.dfs = [self.dfs[1], self.dfs[2]] if self.filter_ind==True else self.dfs
#         self.dfs = [i for i in self.dfs if '央行及金融同業存款' not in i.columns] \
#         if self.filter_ind==True else self.dfs
        self.dfs[0].rename(columns={'收益':'營業收入',
                          '營業利益':'營業利益(損失)',
                          '稅前淨利（淨損）':'稅前淨利(淨損)',
                          '本期淨利（淨損）':'本期淨利(淨損)'}, inplace=True)
        self.dfs[1].rename(columns={'營業利益（損失）':'營業利益(損失)',
                          '稅前淨利（淨損）':'稅前淨利(淨損)',
                          '本期淨利（淨損）':'本期淨利(淨損)'}, inplace=True)
        for i in self.dfs:
            i.rename(columns={'基本每股盈餘（元）':'EPS'}, inplace=True)
        self.dfs = [i[['公司代號', '公司名稱', '營業收入', '營業利益(損失)', '稅前淨利(淨損)', '本期淨利(淨損)', \
                  'EPS']] for i in self.dfs]
        df_all = pd.concat(self.dfs, ignore_index=True)
        self.data = df_all
        
#     資產負債表
    def unify_baln(self):
        self.dfs = [i for i in self.raw_data if '公司代號' in i.columns]
        self.dfs = [i for i in self.dfs if '央行及金融同業存款' not in i.columns] \
            if self.filter_ind==True else self.dfs
        self.dfs[0].rename(columns={'保留盈餘（或累積虧損）':'保留盈餘'
                  ,'資產合計':'資產總額'
                  ,'負債合計':'負債總額'
                  ,'權益合計':'權益總額'}, inplace=True)
        imp = ['公司代號', '公司名稱', '流動資產', '非流動資產', '資產總額', '流動負債', '非流動負債', '負債總額', '股本',
            '資本公積', '保留盈餘', '其他權益', '庫藏股票', '權益總額', '每股參考淨值']
        self.dfs = [i[imp] for i in self.dfs]   
        df_all = pd.concat(self.dfs, ignore_index=True)
        df_all = df_all[df_all['公司代號']!='公司代號']
        self.data = df_all
        
#     營益分析
    def unify_incm_ratio(self):
        dfs = [i for i in x.raw_data if len(i)>0]
        dfs = [i for i in dfs if i.iloc[0][0]=='公司代號']
        data = dfs[0]
        data.columns = list(dfs[0].iloc[0])
        data = data[data['公司代號']!='公司代號']
        data.rename(columns={'毛利率(%)(營業毛利)/(營業收入)':'毛利率(%)'
                     ,'營業利益率(%)(營業利益)/(營業收入)':'營業利益率(%)'
                     ,'稅前純益率(%)(稅前純益)/(營業收入)':'稅前純益率(%)'
                     ,'稅後純益率(%)(稅後純益)/(營業收入)':'稅後純益率(%)'}, inplace=True)
        data = data.reset_index().drop('index', axis=1)
        self.data = data

#     財務結構分析
    def unify_baln_ratio(self, smp=True):
        dfs = [i for i in x.raw_data if i.shape[1]>15]
        data = dfs[0]
        data.columns = data.columns.get_level_values(1)
        data = data[data['公司代號']!='公司代號']
        data = data.reset_index().drop('index', axis=1)
        if smp==True :
            clms = list(data.columns)
            clms = [i for i in clms if i not in \
                   ['長期資金佔不動產、廠房及設備比率(%)', '不動產、廠房及設備週轉率(次)'
                    , '平均收現日數', '純益率(%)', '平均銷貨日數', '稅前純益佔實收資本比率(%)']]
            data = data[clms]
            data.rename(columns={'負債佔資產比率(%)':'負債比率(%)'}, inplace=True)
        self.data = data

# ifrs後

$ 上市 / 損益表 $

In [50]:
x = TwStock(2015, 1, '上市', '損益表')

In [51]:
x.add_raw()

In [52]:
x.unify_incm()

In [54]:
x.data.head()

,公司代號,公司名稱,營業收入,營業利益(損失),稅前淨利(淨損),本期淨利(淨損),EPS
0,2855,統一證,1131274,45846,645816,558741,0.42
1,6005,群益證,1644903,419747,681728,590506,0.24
2,6024,群益期,198086,-88102,70877,57577,0.49
3,1101,台泥,21440142,1693508,1469867,974518,0.22
4,1102,亞泥,15362530,1037375,1844700,1542288,0.47


In [174]:
x = TwStock(2015, 1, '上櫃', '損益表')
x.add_raw()

In [175]:
[i.shape for i in x.dfs]

[(6, 22), (665, 28), (1, 22), (1, 18)]

In [176]:
x.unify_col()

KeyError: "['Net_Income', 'Pre_Tax_Income', 'Operating_Income', 'Gross_Profit'] not in index"

In [178]:
[i.shape for i in x.dfs]

[(6, 22), (665, 28), (1, 22), (1, 18)]

# 財務比率分析

In [39]:
x = TwStock(2015, 1, '上市', '營益分析')
x.add_raw()

In [40]:
x.unify_incm_ratio()

In [41]:
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1101,台泥,21440.14,13.34,7.90,6.86,4.55
1,1102,亞泥,15362.53,10.62,6.75,12.01,10.04
2,1103,嘉泥,671.36,-11.27,-27.67,-31.21,-27.30
3,1104,環泥,1327.43,8.30,1.85,20.32,19.73
4,1108,幸福,1290.90,20.88,15.37,14.48,12.01


In [51]:
x = TwStock(2015, 1, '上櫃', '營益分析')
x.add_raw()

In [52]:
x.unify_incm_ratio()

In [53]:
x.data.head()

,公司代號,公司名稱,營業收入(百萬元),毛利率(%),營業利益率(%),稅前純益率(%),稅後純益率(%)
0,1258,其祥-KY,324.18,32.33,10.06,11.79,9.26
1,1259,安心,1053.39,25.79,1.75,2.16,1.51
2,1264,德麥,937.93,36.67,15.31,15.33,12.41
3,1333,恩得利,226.08,17.71,-3.56,2.64,1.81
4,1336,台翰,418.88,2.63,-10.71,-9.81,-8.17


In [63]:
x = TwStock(2015, 1, '上市', '財務結構分析')
x.add_raw()

In [64]:
x.unify_baln_ratio()

In [65]:
x.data.head()

,公司代號,公司簡稱,負債比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),存貨週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1101,台泥,46.95,133.06,119.28,6.07,4.01,8.15,0.32,2.85,4.36,1.56,31.97,109.73,3.19
1,1102,亞泥,42.66,81.02,69.80,5.14,3.55,7.08,0.24,2.28,3.12,1.55,20.95,112.25,2.97
2,1103,嘉泥,48.70,699.93,609.52,186.35,6.34,1.75,0.10,1.31,1.28,0.34,19.88,59.60,0.24
3,1104,環泥,19.88,123.51,103.82,101.84,4.26,13.93,0.26,7.09,8.70,2.22,34.43,59.18,1.09
4,1108,幸福,38.32,285.67,81.91,15.46,4.91,1.16,0.64,6.14,9.78,1.16,55.80,221.19,4.35


In [16]:
x = TwStock(2015, 1, '上櫃', '財務結構分析')
x.add_raw()

In [17]:
x.unify_baln_ratio()

In [18]:
x.data.head()

,公司代號,公司簡稱,負債佔資產比率(%),長期資金佔不動產、廠房及設備比率(%),流動比率(%),速動比率(%),利息保障倍數(%),應收款項週轉率(次),平均收現日數,存貨週轉率(次),...,不動產、廠房及設備週轉率(次),總資產週轉率(次),資產報酬率(%),權益報酬率(%),稅前純益佔實收資本比率(%),純益率(%),每股盈餘(元),現金流量比率(%),現金流量允當比率(%),現金再投資比率(%)
0,1258,其祥-KY,50.33,183.27,205.87,192.60,24.73,10.87,33.57,47.45,...,4.03,1.48,10.69,17.60,45.76,6.91,3.79,52.33,NaN,14.04
1,1259,安心,36.50,469.99,229.18,222.53,63061.00,80.98,4.50,73.13,...,11.38,1.75,3.41,5.27,38.93,1.95,4.05,42.80,127.72,0.09
2,1264,德麥,22.55,360.49,346.51,263.74,5465.08,4.75,76.84,5.29,...,6.52,1.47,16.71,22.01,175.90,11.19,13.33,70.76,103.33,5.29
3,1268,漢來美食,56.34,188.81,114.89,46.52,37.11,34.01,10.73,31.50,...,4.82,1.19,8.45,21.94,75.23,6.26,6.13,32.11,195.33,17.97
4,1333,恩得利,56.60,449.69,91.01,66.98,-0.02,2.30,158.69,5.27,...,7.95,0.72,-1.24,-6.76,-2.74,-4.00,-0.33,1.86,42.53,1.92


In [20]:
x.data.columns

Index(['公司代號', '公司簡稱', '負債佔資產比率(%)', '長期資金佔不動產、廠房及設備比率(%)', '流動比率(%)',
       '速動比率(%)', '利息保障倍數(%)', '應收款項週轉率(次)', '平均收現日數', '存貨週轉率(次)', '平均銷貨日數',
       '不動產、廠房及設備週轉率(次)', '總資產週轉率(次)', '資產報酬率(%)', '權益報酬率(%)',
       '稅前純益佔實收資本比率(%)', '純益率(%)', '每股盈餘(元)', '現金流量比率(%)', '現金流量允當比率(%)',
       '現金再投資比率(%)'],
      dtype='object')

# balance

In [28]:
r = requests.post('https://mops.twse.com.tw/mops/web/ajax_t51sb08', {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',
        'year':97,
        'season':'01',
    })
r.encoding = 'utf8'
dfs = pd.read_html(StringIO(r.text))

In [31]:
dfs

[                                                   0
 0               註：依證券交易法第36條及證券期貨局相關函令規定，財務報告申報期限如下：
 1  1.個別財務報表申報期限：年度為3月31日（101年申報100年度財務報告起適用），第一季為...
 2  2.合併財務報表申報期限：年度為3月31日（101年申報100年度財務報告起適用），第一季為...
 3                           3.申報期限如遇例假日，以證券期貨局公布者為準。
 4                                           單位：新台幣仟元,
      公司代號  公司名稱      營業收入      營業成本  營業毛利(毛損)  聯屬公司間未實現利益  聯屬公司間已實現利益  \
 0    1101    台泥   5959385   5558941    400444           0         307   
 1    1102    亞泥   2638563   2148067    490496        9093           0   
 2    1103    嘉泥    618302    537922     80380           0           0   
 3    1104    環泥    699474    650364     49110           0           0   
 4    1108    幸福    807109    705057    102052           0           0   
 5    1109    信大    456279    355626    100653         221         631   
 6    1110    東泥    455435    381808     73627           0           0   
 7    1201    味全   2404611   1522446    882165           0           0   
 8    120

In [30]:
def raw_baln(year, season, mkt_type='sii'):
    url = 'https://mops.twse.com.tw/mops/web/t163sb05'
    r = requests.post(url, {
            'encodeURIComponent':1,
            'step':1,
            'firstin':1,
            'off':1,
            'TYPEK':mkt_type,
            'year':year,
            'season':season,
        })
    r.encoding = 'utf8'
    return r.text

In [71]:
dfs = pd.read_html(raw_baln(103, 4))

In [72]:
dfs = [i for i in dfs if i.shape[1] > 15]

In [73]:
[i.shape for i in dfs]

[(11, 53), (3, 21), (855, 22), (14, 54), (7, 46), (4, 21)]

In [48]:
dfs = [i for i in dfs if (i.shape[1] > 15) & (i.shape[1] < 30)]

In [67]:
dfs[0].rename(columns={'保留盈餘（或累積虧損）':'保留盈餘'
                      ,'資產合計':'資產總額'
                      ,'負債合計':'負債總額'
                      ,'權益合計':'權益總額'}, inplace=True)

In [68]:
set.intersection(*map(set,[list(i) for i in dfs]))

{'保留盈餘',
 '公司代號',
 '公司名稱',
 '共同控制下前手權益',
 '其他權益',
 '庫藏股票',
 '待註銷股本股數（單位：股）',
 '權益總額',
 '每股參考淨值',
 '流動負債',
 '流動資產',
 '股本',
 '負債總額',
 '資本公積',
 '資產總額',
 '非流動負債',
 '非流動資產',
 '預收股款（權益項下）之約當發行股數（單位：股）'}

In [70]:
imp = ['公司代號', '公司名稱', '流動資產', '非流動資產', '資產總額', '流動負債', '非流動負債', '負債總額', '股本',
        '資本公積', '保留盈餘', '其他權益', '庫藏股票', '權益總額', '每股參考淨值']

In [71]:
dfs = [i[imp] for i in dfs]

In [73]:
def unify_baln(x):
    dfs = x.copy()
    dfs[0].rename(columns={'保留盈餘（或累積虧損）':'保留盈餘'
                      ,'資產合計':'資產總額'
                      ,'負債合計':'負債總額'
                      ,'權益合計':'權益總額'}, inplace=True)
    imp = ['公司代號', '公司名稱', '流動資產', '非流動資產', '資產總額', '流動負債', '非流動負債', '負債總額', '股本',
        '資本公積', '保留盈餘', '其他權益', '庫藏股票', '權益總額', '每股參考淨值']
    dfs = [i[imp] for i in dfs]   
    df_all = pd.concat(dfs, ignore_index=True)
    return df_all

In [74]:
df_all = unify_baln(dfs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [78]:
def get_baln(year, season):
    year = year if year < 1000 else year-1911
#     if year >= 1000:
#         year -= 1911
    dfs = pd.read_html(raw_baln(year, season))
    dfs = [i for i in dfs if (i.shape[1] > 15) & (i.shape[1] < 30)]
    # 將不同欄位名統一
    df_all = unify_baln(dfs)    
    return df_all

In [77]:
year = 1111111111
year = year if year < 1000 else year-1911
print(year)

1111109200


In [79]:
get_baln(103, 4)

,公司代號,公司名稱,流動資產,非流動資產,資產總額,流動負債,非流動負債,負債總額,股本,資本公積,保留盈餘,其他權益,庫藏股票,權益總額,每股參考淨值
0,2855,統一證,59972212,4975334,64947546,41811986,60084,41872070,13231191,256116,9431778,113539,--,23075476,17.41
1,6005,群益證,69531529,14156814,83688343,49996260,1489963,51486223,23690730,2711760,4598641,97349,0,32202120,13.13
2,6024,群益期,14958003,476365,15434368,12730741,11195,12741936,1199979,377709,1062107,52637,0,2692432,22.44
3,1101,台泥,96605816,196138016,292743832,68632044,62094107,130726151,36921759,12225528,49530227,19281356,--,162017681,31.95
4,1102,亞泥,70558875,212260180,282819055,64989203,56102626,121091829,33614472,1073920,94863921,12281251,--,161727226,42.19
5,1103,嘉泥,12878015,20788464,33666479,3366245,11018176,14384421,7768285,570096,8390425,1774583,-1149426,19282058,26.88
6,1104,環泥,2652681,16730075,19382756,2068617,1516169,3584786,6159697,28999,7993017,1460564,--,15797970,25.39
7,1108,幸福,5561570,2801709,8363279,2333817,1414348,3748165,4047380,0,508990,28252,0,4615114,11.33
8,1109,信大,2767192,6516323,9283515,1840346,411048,2251394,4210008,22299,1568770,167234,0,7032121,14.18
9,1110,東泥,1788502,7814235,9602737,294502,448910,743412,5720008,184916,1983703,948426,-12185,8859325,15.49
